![](https://lh3.googleusercontent.com/qbqjDFiK6wInTH4KrDaNIxbvng1XMs2F7Cv_DbzZcm-ljDA0Ikyp9y4HxOLIhp2r_ks=s180-rw?v=0.1?raw=true)
![](https://img-d02.moneycontrol.co.in/images/common/header/logo.png?v=0.1?raw=true)

# Moneycontrol scraping
### First we need to scrape all the companies under Indian Market

In [1]:
from IPython.display import display, HTML
from multiprocessing.dummy import Pool
from bs4 import BeautifulSoup
import pandas as pd
import requests,json

#### The function which scraps links like:

https://www.moneycontrol.com/india/stockpricequote/A  
https://www.moneycontrol.com/india/stockpricequote/B  
.
.
.  
https://www.moneycontrol.com/india/stockpricequote/Z  
https://www.moneycontrol.com/india/stockpricequote/others

#### You can refer to this [Stack Overflow answer](https://stackoverflow.com/a/55046746/8141330) to get an idea how to get sc_id from any stock, which we will require to get historical prices later


In [90]:
company_list=[]

# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# def get_sc(company_name):
#     from selenium import webdriver

#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver.get("https://www.moneycontrol.com/stocks/histstock.php?")

#     search_input_box = driver.find_element_by_id("mycomp")
#     search_input_box.send_keys(company_name)
#     print(driver.find_element_by_text(search_input_box))

'''
def get_sc_id(company_name):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'}
    # Enter search text
    query_input = company_name
    #Get suggested sc_id
    suggest_query_url = 'https://www.moneycontrol.com/mccode/common/autosuggestion_solr.php'
    query = {
    'classic': 'true',
    'query': query_input,
    'type': '1',
    'format': 'json',
    'callback': 'suggest1'}
    # Pull out the sc_id
    suggested_response = requests.get(suggest_query_url , headers=headers, params=query).text
    print("\n\n\n\n",company_name,suggested_response.split('\"sc_id\":\"')[1],"\n\n\n\n")
#     suggested_response = suggested_response.split('(',1)[1]
#     suggested_response = suggested_response.rsplit(')',1)[0]
#     sc_id = json.loads(suggested_response)[0]['sc_id']
#     return(sc_id)
'''

def get_company_list(url):
    per_row=[]
    page_response = requests.get(url, timeout=5)
    soup = BeautifulSoup(page_response.content,"lxml")
    companies = soup.findAll("a", attrs={"class":"bl_12"})
    for company in companies:
        if "moneycontrol" in company["href"]:
#             sc_id = get_sc_id(company.text)
#             print([company.text,company["href"],sc_id])
            per_row.append(company.text)
            per_row.append(company['href'])
#             company_list.append([company.text,company["href"]])
        for li in soup.findAll("table",attrs={"class":"comInfo"}):
            for a in (li.findAll("a")):
                per_row.append("https://www.moneycontrol.com"+a["href"])
        company_list.append(per_row)
#     print(company_list)
# get_company_list("https://www.moneycontrol.com/india/stockpricequote/A")


#### This generates all links and sends to the function through multithreading

In [91]:
urls=[]
incomplete_url="https://www.moneycontrol.com/india/stockpricequote/"
for i in range (ord("A"),ord("Z")+1):
    urls.append(incomplete_url+chr(i))
urls.append((incomplete_url+"others"))
with Pool(processes=27) as pool:
    (pool.map(get_company_list, urls))
df =  pd.DataFrame(company_list)
column_list=['Company Names', 'Company Links']
any_page=requests.get("https://www.moneycontrol.com/india/stockpricequote/electric-equipment/igarashimotors/IM01", timeout=5)
soup=BeautifulSoup(any_page.content,"lxml")
for li in soup.findAll("table",attrs={"class":"comInfo"}):
    for a in (li.findAll("a")):
        column_list.append([a.text])
# df.columns = ['Company Names', 'Company Links']
df.columns = column_list
df.to_csv('company_list.csv', sep=',', header=None, index=None)

ReadTimeout: HTTPSConnectionPool(host='www.moneycontrol.com', port=443): Read timed out. (read timeout=5)

#### This is the table (only part of it)

In [8]:
print ("Dataframe 1:")
df[:10]

Dataframe 1:


,Company Names,Company Links
0,Xchanging Sol,http://www.moneycontrol.com/india/stockpricequ...
1,Xedd Telecom,http://www.moneycontrol.com/india/stockpricequ...
2,Xeddtelcom,http://www.moneycontrol.com/india/stockpricequ...
3,XL Energy,http://www.moneycontrol.com/india/stockpricequ...
4,XLO-Machine,http://www.moneycontrol.com/india/stockpricequ...
5,XLO-United Clut,http://www.moneycontrol.com/india/stockpricequ...
6,Xo - Tronics,http://www.moneycontrol.com/india/stockpricequ...
7,Xpro India,http://www.moneycontrol.com/india/stockpricequ...
8,Xylon Loquitur,http://www.moneycontrol.com/india/stockpricequ...
9,Quadrant Tele,http://www.moneycontrol.com/india/stockpricequ...


#### Now get a perticular company and get the link to that Stock Website

In [10]:
company_name_given_by_user = "igarashi motors"
company_link=df[df["Company Names"].str.contains(company_name_given_by_user, case=False)]["Company Links"].iloc[0]
# sc_id=df[df["Company Names"].str.contains(company_name_given_by_user, case=False)]["sc_id"].iloc[0]
print("The link: "+company_link)
# print("sc_id: "+sc_id)

The link: http://www.moneycontrol.com/india/stockpricequote/electricequipment/igarashimotors/IM01


### Now get live BSE, NSE data from the user given website

In [11]:
page_response = requests.get(company_link, timeout=5)
soup = BeautifulSoup(page_response.content,"lxml")

#### Get BSE data

In [12]:
bse_live = soup.findAll("span", attrs={"id":"Bse_Prc_tick"})[0].text
bse_change = soup.findAll("div", attrs={"id":"b_changetext"})[0].text
bse_change_volume = bse_change.split("(")[0]
bse_change_percentage = bse_change.split("(")[1].split(")")[0]
print("Live BSE: "+bse_live+" | BSE change: "+bse_change_volume+" | BSE change percentage: "+bse_change_percentage)

Live BSE: 182.40 | BSE change:  1.50  | BSE change percentage: +0.83%


#### Get NSE data

nse_live = soup.findAll("span", attrs={"id":"Nse_Prc_tick"})[0].text
nse_change = soup.findAll("div", attrs={"id":"n_changetext"})[0].text
nse_change_volume = nse_change.split("(")[0]
nse_change_percentage = nse_change.split("(")[1].split(")")[0]
print("Live NSE: "+nse_live+" | NSE change: "+nse_change_volume+" | NSE change percentage: "+nse_change_percentage)

In [94]:
lista=[]
for li in soup.findAll("table",attrs={"class":"comInfo"}):
    for a in (li.findAll("a")):
        print([a.text,"https://www.moneycontrol.com"+a["href"]])
        lista.append([a.text,"https://www.moneycontrol.com"+a["href"]])
# print((lista))

['Business', 'https://www.moneycontrol.com/news/igarashimotors/business-IM01-6months.html#IM01']
['Earnings', 'https://www.moneycontrol.com/news/igarashimotors/results-IM01-6months.html#IM01']
['Mgmt Interviews ', 'https://www.moneycontrol.com/news/igarashimotors/management-interviews-IM01-6months.html#IM01']
['Announcements', 'https://www.moneycontrol.com/company-notices/igarashimotors/notices/IM01#IM01']
['Stock Views', 'https://www.moneycontrol.com/news/igarashimotors/stock-views-IM01-6months.html#IM01']
['Brokerage Reports', 'https://www.moneycontrol.com/news/igarashimotors/brokerage-reports-IM01-6months.html#IM01']
['Sector', 'https://www.moneycontrol.com/stocks/company_info/sector_info.php?sub_sector=Electric Equipment#IM01']
['Board Meetings', 'https://www.moneycontrol.com/company-facts/igarashimotors/board-meetings/IM01#IM01']
['AGM/EGM', 'https://www.moneycontrol.com/company-facts/igarashimotors/agm-egm/IM01#IM01']
['Bonus', 'https://www.moneycontrol.com/company-facts/igarashi